In [ ]:
# Install required packages
!pip install flask transformers torch pillow flask-cors pyngrok accelerate bitsandbytes

# Configure ngrok authentication
from pyngrok import ngrok, conf
import os

# Set auth token
NGROK_AUTH_TOKEN = "2oHUyMyGNJuD34GO6NdGJd8KAxd_3TyYzUGLMA9DvgUopNRw3"

# Clear any existing ngrok configurations
!rm -rf ~/.ngrok2
!rm -rf ~/.config/ngrok

# Configure ngrok
conf.get_default().auth_token = NGROK_AUTH_TOKEN
ngrok.kill() # Kill any existing ngrok processes

print("Ngrok authentication configured")


: 

In [0]:
# Start Flask server with ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from PIL import Image
import io

app = Flask(__name__)
CORS(app)

try:
    # Set up ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f'Ngrok tunnel established! Public URL: {public_url}')
    
    # Load AI models with GPU support
    print('Loading AI models...')
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load image captioning model
    image_captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base", device=device)
    
    # Load story generation model
    story_model = "HuggingFaceH4/zephyr-7b-beta"
    story_generator = pipeline(
        "text-generation",
        model=story_model,
        torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
        device_map="auto",
        max_length=1000
    )
    
    print(f'Models loaded successfully on {device}!')
    
    def generate_horror_story(caption):
        """Generate a horror story using Zephyr-7B"""
        prompt = f"""<|system|>You are a master of horror stories, skilled at creating atmospheric and spooky tales.

<|user|>Write a spooky horror story based on this scene: {caption}
Requirements:
- Story should be 200-300 words long
- Include atmospheric details and build tension
- Have a twist or surprising ending
- Keep it suitable for young adults
- Use descriptive language to set the mood

<|assistant|>"""
        
        result = story_generator(
            prompt,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            return_full_text=False
        )[0]['generated_text']
        
        return result.strip()
    
    @app.route('/generate_story', methods=['POST'])
    def generate_story():
        try:
            if 'image' not in request.files:
                return jsonify({'status': 'error', 'message': 'No image provided'}), 400
                
            image_file = request.files['image']
            image = Image.open(io.BytesIO(image_file.read()))
            
            # Generate caption from image
            caption = image_captioner(image)[0]['generated_text']
            
            # Generate horror story based on caption
            story = generate_horror_story(caption)
            
            return jsonify({
                'status': 'success',
                'caption': caption,
                'story': story
            })
            
        except Exception as e:
            return jsonify({'status': 'error', 'message': str(e)}), 500

except Exception as e:
    print(f"Error setting up server: {str(e)}")
    ngrok.kill()
    raise e

if __name__ == '__main__':
    # Run app with host='0.0.0.0' to allow external connections
    app.run(host='0.0.0.0', port=5000)
